## Выполнил: Сиволобцев Роман

# Импорт

In [1]:
from __future__ import division, unicode_literals

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score
from sklearn import cross_validation

%matplotlib inline

In [2]:
train = pd.read_csv('train.csv', encoding='utf-8', lineterminator='\n')
test = pd.read_csv('test.csv', encoding='utf-8', lineterminator='\n')

# Выбор признаков

In [3]:
for index, field in enumerate(train.columns):
    print(field)

id
text
in_reply_to_user_id
user.id
user.name
user.screen_name
user.description
user.location
user.lang
user.time_zone
user.utc_offset
user.statuses_count
user.followers_count
user.friends_count
user.favourites_count
user.created_at
user.geo_enabled
user.is_translation_enabled
user.listed_count
retweet_count


Данных о твит-сообщениях много, для начала уберём ненужную информацию: 
- Имя пользователя. Запоминание конкретных имён может привести к переобучению, а учитывая что авторы сообщений в тестовых и обучающих выборках не пересекаются, получаем избыточный признак.
- Временную зону. Так как статистика идёт за 48 часов твита и нет разницы в какое время его написали.
- Дату создания аккаунта. Каждый уделяет столько времени сколько он хочет - кто-то больше, кто-то меньше. А некоторые люди могут создать аккаунт, закинуть его, а потом снова вспомнить. На мой взгляд этот признак рассматривать не нужно.

In [4]:
del train['user.name']
del train['user.utc_offset']
del train['user.time_zone']
del train['user.created_at']

train[:5]

,id,text,in_reply_to_user_id,user.id,user.screen_name,user.description,user.location,user.lang,user.statuses_count,user.followers_count,user.friends_count,user.favourites_count,user.geo_enabled,user.is_translation_enabled,user.listed_count,retweet_count
0,629692043326062592,Me and ma fwends 🍎 http://t.co/B3YJ31hZuc,0.0,133840449,xcaptainpaulx,Guitarist for @chunknocaptainc // Founder @off...,NaN,fr,1606,9164,205,758,True,False,27,12
1,629692041362968576,@SinedioMD @AspyrMedia Try this link https://t...,7629232.0,21245956,Direct2Drive,D2D has an ever-expanding library of downloada...,"California, USA",en,3004,7484,708,44,False,False,330,0
2,629692040679419904,@davidrobots @GamesRadar @David_H_Esq @CatGone...,16535125.0,63785369,ajohnagnello,I am the Senior Social Editor at GamesRadar+ a...,New York City,en,3988,2050,491,422,False,False,70,0
3,629692040666812416,Already hearing scanner traffic from Heath ask...,0.0,28812126,bethany_bruner,"Breaking News Reporter for @NewarkAdvocate, @...",NaN,en,9018,919,309,79,True,False,38,0
4,629692038842306560,@MelJohnson6527 next year with @Chuck_Ellis #c...,181232379.0,44024059,NdotSmitty,2010 Duke National Champ! I'm Still ON A MISSI...,DMV/Durham/Louisville/ Next??,en,32527,85356,912,137,True,False,1232,1


В твитах есть ссылки на аккаунты других пользователей, обозначающиеся после символа "@". А если это сами по себе ретвиты, то в in_reply_to_user_id хранится отсылка к аккаунту начального автора твита. Можно сделать как один из признаков отсылку на данные пользователя упомянутого в данном твите. НО! Учитывая что база не полная (существуют ссылки на аккаунты которые не предоставлены в базе) и то что это может вызвать переобучение, было решено отказаться от такого признака. 

При обработке текста, будут учтены ссылки, но не на конкретные объекты, а просто как количественный параметр.
В связи с этим данные в колонках user.id, in_reply_to_user_id, user.screen_name, перестали быть информативными. Удалим или заменим их.

In [5]:
# Перед удалением пользовательских id, отсортируем по ним выборку, для лучшей кросс-валидации в будущем.
train = train.sort_values(by = 'user.id')

del train['user.screen_name']
del train['user.id']

train['in_reply_to_user_id'] = train['in_reply_to_user_id']>0 # реформируем поля, теперь: 1 - ретвит, 0 - твит.

In [6]:
train[:5]

,id,text,in_reply_to_user_id,user.description,user.location,user.lang,user.statuses_count,user.followers_count,user.friends_count,user.favourites_count,user.geo_enabled,user.is_translation_enabled,user.listed_count,retweet_count
30006,629662292427157508,Puerto Rico is the new Greece https://t.co/FD...,False,VC @FirstRound and BD always. Technology histo...,NYC and PHL mostly,en,12954,9692,1692,2662,True,False,549,0
14145,629678103179210752,Surprise 70th bday party /(&amp; 20-person gol...,False,"I like snowboards, foursquare and unemployment.",nyc / kingston,en,45371,79241,1476,2330,True,False,4345,1
8877,629683426694664192,OMG my brother-in-law Brian just hit a 107 yar...,False,"I like snowboards, foursquare and unemployment.",nyc / kingston,en,45371,79241,1476,2330,True,False,4345,0
3791,629688126580002816,Yahoo uses Periscope. https://t.co/TAYrlpXLxn,False,"Married to @crystale, dad to @CCEleven & @Circ...",NaN,en,40407,1616444,867,123811,True,False,10506,8
6093,629685761026166784,Lots of people applaud @realDonaldTrump for ho...,False,"Dir. Audience Development (@TCTotem), author (...","Toronto, ON",en,54026,51351,3307,3895,True,False,3948,2


In [7]:
train.corr()

,id,in_reply_to_user_id,user.statuses_count,user.followers_count,user.friends_count,user.favourites_count,user.geo_enabled,user.is_translation_enabled,user.listed_count,retweet_count
id,1.000000,-0.009327,-0.030182,-0.000007,0.003039,0.016018,-0.000259,-0.007774,0.001763,0.012381
in_reply_to_user_id,-0.009327,1.000000,0.307408,-0.000940,0.071367,0.100630,-0.057485,-0.024408,-0.047139,-0.021664
user.statuses_count,-0.030182,0.307408,1.000000,0.052894,0.127003,0.024958,0.077813,0.040412,0.054768,-0.009976
user.followers_count,-0.000007,-0.000940,0.052894,1.000000,0.136822,0.022280,-0.035867,0.310828,0.845898,0.577299
user.friends_count,0.003039,0.071367,0.127003,0.136822,1.000000,0.179443,0.008857,0.025236,0.099393,0.057979
user.favourites_count,0.016018,0.100630,0.024958,0.022280,0.179443,1.000000,0.080932,0.018860,0.020612,0.000729
user.geo_enabled,-0.000259,-0.057485,0.077813,-0.035867,0.008857,0.080932,1.000000,-0.010995,-0.026231,-0.011232
user.is_translation_enabled,-0.007774,-0.024408,0.040412,0.310828,0.025236,0.018860,-0.010995,1.000000,0.272556,0.089645
user.listed_count,0.001763,-0.047139,0.054768,0.845898,0.099393,0.020612,-0.026231,0.272556,1.000000,0.641381
retweet_count,0.012381,-0.021664,-0.009976,0.577299,0.057979,0.000729,-0.011232,0.089645,0.641381,1.000000


Посмотрим с какими строчками больше всего корелирует retweet_count. Довольно ожидаема вылезла зависимость между user.followers_count и user.listed_count. На данной стадии user.followers_count - главный признак. 

Теперь поработаем с самым главным - текстом твитов. Проверим самые простые признаки, которые из него можно вытянуть - длину сообщения, количество ссылок на другие аккаунты(@Name), количество хэштегов(#), и наличием ссылки на сторонний ресурс.

In [8]:
def symbol_count(symbol, string):
    count = 0
    for i in range(len(string)):
        if symbol == string[i]:
            count+=1 
    return count

def text_params(text):
    param1 = []
    param2 = []
    param3 = []
    param4 = []
    for word in text:
        param1.append(len(str(word))) # длинна текста
        param2.append(symbol_count('@', str(word))) # количество ссылок на другие аккаунты
        param3.append(symbol_count('#', str(word))) # количество хэштегов
        param4.append('http' in str(word)) # есть ли ссылка на сторонний ресурс
        
    return [param1, param2, param3, param4]

In [9]:
params = text_params(train['text'])

train.insert(2, 'text_length', params[0])
train.insert(3, 'text_account_count', params[1])
train.insert(4, 'text_hashtag_count', params[2])
train.insert(5, 'text_is_website_enabled', params[3])

# Построение выборки

In [10]:
train[:5]

,id,text,text_length,text_account_count,text_hashtag_count,text_is_website_enabled,in_reply_to_user_id,user.description,user.location,user.lang,user.statuses_count,user.followers_count,user.friends_count,user.favourites_count,user.geo_enabled,user.is_translation_enabled,user.listed_count,retweet_count
30006,629662292427157508,Puerto Rico is the new Greece https://t.co/FD...,54,0,0,True,False,VC @FirstRound and BD always. Technology histo...,NYC and PHL mostly,en,12954,9692,1692,2662,True,False,549,0
14145,629678103179210752,Surprise 70th bday party /(&amp; 20-person gol...,124,1,0,True,False,"I like snowboards, foursquare and unemployment.",nyc / kingston,en,45371,79241,1476,2330,True,False,4345,1
8877,629683426694664192,OMG my brother-in-law Brian just hit a 107 yar...,110,1,1,True,False,"I like snowboards, foursquare and unemployment.",nyc / kingston,en,45371,79241,1476,2330,True,False,4345,0
3791,629688126580002816,Yahoo uses Periscope. https://t.co/TAYrlpXLxn,45,0,0,True,False,"Married to @crystale, dad to @CCEleven & @Circ...",NaN,en,40407,1616444,867,123811,True,False,10506,8
6093,629685761026166784,Lots of people applaud @realDonaldTrump for ho...,138,2,0,False,False,"Dir. Audience Development (@TCTotem), author (...","Toronto, ON",en,54026,51351,3307,3895,True,False,3948,2


In [11]:
def features(df):
    return np.array([
        df['text_length'],
        df['text_account_count'],
        df['text_hashtag_count'],
        df['user.statuses_count'],
        df['user.followers_count'],
        df['user.friends_count'],
        df['user.favourites_count'],
        df['text_is_website_enabled'],
        df['user.geo_enabled'],
        df['user.is_translation_enabled'],
        df['user.listed_count']
    ]).transpose()

train_X = features(train)
train_Y = np.array(train['retweet_count'] > 20).transpose()

# Кросс-валидация 

В самом начале выборка была отсортирована по пользователям, поэтому используя разбиение KFolds c shuffle=False, мы получим хорошое разбиение на тренировочную и тестовую выборку. Можно было делать разбиение выборки по пользователям (так тренировочные и тестовые не будут пересекаться), но я решил пренебречь возможным пересечением выборок по 1 пользователю с целью сохранить одинаковое процентное соотношение тестовой и обучающей выборки.

In [12]:
def validation(classifier_type, data_X, data_Y):
    from sklearn import cross_validation
    
    scorer = []
    validation = cross_validation.KFold(len(data_X), n_folds=5, shuffle=False, random_state=1)
    for train_index, test_index in validation:
        classifier = classifier_type()
        classifier.fit(data_X[train_index], data_Y[train_index])
        proba = classifier.predict_proba(data_X[test_index])
        test = classifier.predict_proba(data_X[train_index])
        scorer.append([roc_auc_score(data_Y[test_index], proba[:,1]),roc_auc_score(data_Y[train_index], test[:,1])])
    return scorer

# Предварительная оценка первой модели

In [13]:
from sklearn.tree import DecisionTreeClassifier

scorer = np.array(validation(DecisionTreeClassifier, train_X, train_Y))
print([scorer[:,0].mean(), scorer[:,0].std(),scorer[:,1].mean(), scorer[:,1].std()])
scorer

[0.64592619340601254, 0.038774771368768356, 0.99998748261545722, 1.3023880159233403e-06]


array([[ 0.71849201,  0.99998913],
       [ 0.63185754,  0.99998661],
       [ 0.64257711,  0.99998734],
       [ 0.63450483,  0.99998562],
       [ 0.60219947,  0.99998871]])

Первая колонка - значения на тестовой выборке, вторая - на обучающей.

Вывод: если использовать метод из демоверсии, то получается просто ужаснейшее переобучение. Используем другой классификатор.

In [14]:
from sklearn.ensemble import GradientBoostingClassifier

scorer = np.array(validation(GradientBoostingClassifier, train_X, train_Y))
print([scorer[:,0].mean(), scorer[:,0].std(),scorer[:,1].mean(), scorer[:,1].std()])
scorer

[0.89047031756003903, 0.017928058330680632, 0.92153204565500935, 0.0021845700617367687]


array([[ 0.91186907,  0.91799599],
       [ 0.88807341,  0.92235506],
       [ 0.90411234,  0.9207921 ],
       [ 0.85957943,  0.92470277],
       [ 0.88871735,  0.92181431]])

Дела обстаят более реалестично. Но попробуем сделать ещё лучше.

# Продолжение исследований признаков

In [15]:
train['user.lang'].value_counts()

en                    65971
en-gb                   563
fr                      146
es                      139
de                       96
it                       82
nl                       75
ja                       60
ru                       23
pt                       16
tr                       13
en-GB                     5
ko                        4
sv                        4
ca                        4
pl                        2
Select Language...        1
uk                        1
da                        1
fi                        1
no                        1
xx-lc                     1
zh-cn                     1
ar                        1
Name: user.lang, dtype: int64

Подавляющее преимущество тэгов "en". Проверим есть ли существенное отличие между группами. Но для начала опустим языки, в которых меньше 10 представителей.

In [16]:
def param_popular(param, param_count, pop):
    data = []
    for i in param: 
        if param_count[i]>pop:
            data.append(i)
    return data

def param_del(param, data_param):
    data = []
    for i in data_param:
        if i in param:
            data.append(i)
        else:
            data.append('unimportante')
    return data
    

lang = param_popular(train['user.lang'].unique(), train['user.lang'].value_counts(), 10) # удалим редковстречающиеся языки
train['user.lang'] = param_del(lang, train['user.lang']) # заменим такие языки на 'unimportante'
lang.append('unimportante')
train['user.lang'].value_counts()

en              65971
en-gb             563
fr                146
es                139
de                 96
it                 82
nl                 75
ja                 60
unimportante       27
ru                 23
pt                 16
tr                 13
Name: user.lang, dtype: int64

In [17]:
test['user.lang'] = param_del(lang, test['user.lang']) # тоже самое для тестовой выборки

In [18]:
def over20(data):
    return data[data['retweet_count']>20]

def over20_percent(data):
    return len(data[data['retweet_count']>20])/len(data)

In [19]:
# Посмотрим есть ли минимальная предрасположенность
lang_param = {lang[i]: over20_percent(train[train['user.lang'] == lang[i]]) for i in range(len(lang))}
lang_param

{'de': 0.08333333333333333,
 'en': 0.06398265904715708,
 'en-gb': 0.06216696269982238,
 'es': 0.10071942446043165,
 'fr': 0.06164383561643835,
 'it': 0.13414634146341464,
 'ja': 0.11666666666666667,
 'nl': 0.10666666666666667,
 'pt': 0.25,
 'ru': 0.08695652173913043,
 'tr': 0.23076923076923078,
 'unimportante': 0.18518518518518517}

In [20]:
train['user.lang'] = [lang_param[i] for i in train['user.lang']] # заменяем языки на численные значения
test['user.lang'] = [lang_param[i] for i in test['user.lang']] # заменяем языки на численные значения

Так как не важных признаков получилось не много (27 и 34 в тренировочной и тестовой выборках соответственно), то можно использовать как признак: средний процент сообщений с больше чем 20-ью ретвитами по данному языку. 

Посмотрим какая ситуация обстоит с местоположением.

In [21]:
train_locations = train['user.location'].unique()
test_locations = test['user.location'].unique()
locations_inter = [val for val in train_locations if val in test_locations]
[len(train_locations), len(test_locations), len(locations_inter)]

[5680, 5654, 1223]

Пересечение очень слабое, попробуем убрать непопулярные места и снова посмотреть пересечение.

In [22]:
train['user.location'] = train['user.location'].astype(object).replace(np.nan, 'None') # заменем обекты Nan на текст None
test['user.location'] = test['user.location'].astype(object).replace(np.nan, 'None') # и в тестовой выборке

train_loc = param_popular(train['user.location'].unique(), train['user.location'].value_counts(), 20) 
train['user.location'] = param_del(train_loc, train['user.location']) 
train_loc.append('unimportante')        #замена непопулярных локаций на unimportant

test_loc = param_popular(test['user.location'].unique(), test['user.location'].value_counts(), 20) 
test_loc.append('unimportante')

locations_popular_inter_ = [val for val in train_loc if val in test_loc]
[len(train_loc), len(test_loc), len(locations_popular_inter_)]

[388, 400, 177]

In [23]:
# процент пустых и малозначимых полей
100*(train['user.location'].value_counts()['unimportante']+train['user.location'].value_counts()['None'])/len(train['user.location'])

44.818556486289445

Получается, что только чуть больше половины выборки имеет информативную ценность, и чуть меньше половины этих информативных данных встретятся в тестовой выборке. Насколько этот признак будет хорош - сказать трудно, лучше посмотреть, как будет работать классификатор с ним и без него. Но в обучающую выборку мы его занесём

In [24]:
loc_param = {train_loc[i]: over20_percent(train[train['user.location'] == train_loc[i]]) for i in range(len(train_loc))}
train['user.location'] = [loc_param[i] for i in train['user.location']]

In [25]:
test['user.location'] = param_del(train_loc, test['user.location']) # не забываем про тестовую
test['user.location'] = [loc_param[i] for i in test['user.location']]

Попробуем, покопаться в текстовом поле user.description. Вытянем из него те же признаки как для текстового поля сообщения. Есть подозрения что в этом поле нету никаких информативных признаков.

In [26]:
params = text_params(train['user.description'])

train.insert(6, 'user.description_length', params[0])
train.insert(7, 'user.description_account_count', params[1])
train.insert(8, 'user.description_hashtag_count', params[2])
train.insert(9, 'user.description_is_website_enabled', params[3])

In [27]:
del train['user.description']
del train['text']
del train['id']

train[:5]

,text_length,text_account_count,text_hashtag_count,text_is_website_enabled,user.description_length,user.description_account_count,user.description_hashtag_count,user.description_is_website_enabled,in_reply_to_user_id,user.location,user.lang,user.statuses_count,user.followers_count,user.friends_count,user.favourites_count,user.geo_enabled,user.is_translation_enabled,user.listed_count,retweet_count
30006,54,0,0,True,74,1,0,False,False,0.06445,0.063983,12954,9692,1692,2662,True,False,549,0
14145,124,1,0,True,47,0,0,False,False,0.06445,0.063983,45371,79241,1476,2330,True,False,4345,1
8877,110,1,1,True,47,0,0,False,False,0.06445,0.063983,45371,79241,1476,2330,True,False,4345,0
3791,45,0,0,True,160,7,0,True,False,0.07655,0.063983,40407,1616444,867,123811,True,False,10506,8
6093,138,2,0,False,155,5,0,True,False,0.06000,0.063983,54026,51351,3307,3895,True,False,3948,2


# Повторное построение классификатора

In [28]:
signs = train.columns.values # вытягиваем название колонок 
train_Y = np.array(train[signs[-1]]>20) 
train_X = np.array(train[signs[:-1]])

In [29]:
scorer = np.array(validation(GradientBoostingClassifier, train_X, train_Y))
print([scorer[:,0].mean(), scorer[:,0].std(), scorer[:,1].mean(), scorer[:,1].std()])
scorer

[0.91539813876952469, 0.0068799962742319568, 0.93537530537293212, 0.0014590886575695571]


array([[ 0.92848161,  0.93280493],
       [ 0.91053513,  0.93723292],
       [ 0.91624223,  0.93551718],
       [ 0.91102409,  0.93611023],
       [ 0.91070763,  0.93521128]])

Теоретически результаты удалось поднять на 1.7%. Попробуем поочерёдно выкидывать по признаку и тестировать модель снова, возможно мы имеем неинформативный признак приводящий к обучению.

In [30]:
def del_param(X, Y, signs, classifier):
    answer = pd.DataFrame(columns = ['id','signs', 'test.mean()', 'test.std()', 'train.mean()'])
    scorer = np.array(validation(GradientBoostingClassifier, X, Y))
    answer = answer.append([{'id': 100,'signs': 'full', 'test.mean()': scorer[:,0].mean(), 'train.mean()': scorer[:,1].mean(), 'test.std()': scorer[:,0].std()}])
    train_Y = Y
    for i in range(len(X[1])):
        train_X = np.delete(X, i, 1) # удаляем один признак
        scorer = np.array(validation(classifier, train_X, train_Y))
        answer = answer.append([{'id': i, 'signs': 'without ' + signs[i], 'test.mean()': scorer[:,0].mean(), 'train.mean()': scorer[:,1].mean(), 'test.std()': scorer[:,0].std()}])
    return answer.sort_values(by='test.mean()', ascending=False)

In [32]:
# вычисляется долго, около 15 минут
del_param(train_X, train_Y, signs, GradientBoostingClassifier) 

,id,signs,test.mean(),test.std(),train.mean()
0,13.0,without user.friends_count,0.916466,0.006535,0.935535
0,16.0,without user.is_translation_enabled,0.916243,0.006808,0.935166
0,5.0,without user.description_account_count,0.916235,0.006407,0.935346
0,4.0,without user.description_length,0.916145,0.006635,0.935185
0,7.0,without user.description_is_website_enabled,0.916004,0.006773,0.935277
0,10.0,without user.lang,0.916004,0.006757,0.935205
0,2.0,without text_hashtag_count,0.915981,0.006628,0.935071
0,6.0,without user.description_hashtag_count,0.915771,0.007177,0.935314
0,15.0,without user.geo_enabled,0.915685,0.006915,0.935435
0,100.0,full,0.915541,0.006700,0.935493


Будем повторять данную процедуру до тех пор, пока удаление лишнего параметра не перестанет давать результат.

In [33]:
def del_params(X, Y, signs, classifier):
    answer = del_param(X, Y, signs, classifier)
    while answer.iat[0,1]!='full':
        print(answer.iat[0,1])
        X = np.delete(X, answer.iat[0,0], 1)
        signs = np.delete(signs, answer.iat[0,0], 0)
        answer = del_param(X, Y, signs, classifier)
    return [answer, signs]

In [34]:
# вычисляется ещё дольше, около часа
train_X_del_params = del_params(train_X, train_Y, signs, GradientBoostingClassifier)
train_X_del_params[0]

without user.friends_count
without user.is_translation_enabled
without user.description_account_count


,id,signs,test.mean(),test.std(),train.mean()
0,100.0,full,0.917465,0.006236,0.935033
0,6.0,without user.description_is_website_enabled,0.917432,0.006187,0.935033
0,13.0,without user.geo_enabled,0.917407,0.006144,0.935049
0,5.0,without user.description_hashtag_count,0.917268,0.006394,0.934985
0,9.0,without user.lang,0.917105,0.006526,0.934925
0,4.0,without user.description_length,0.916835,0.006765,0.934646
0,0.0,without text_length,0.916526,0.005973,0.933899
0,12.0,without user.favourites_count,0.916510,0.005619,0.933289
0,2.0,without text_hashtag_count,0.916428,0.006189,0.934841
0,3.0,without text_is_website_enabled,0.916099,0.006317,0.934446


Удалось ещё немного поднять результат на тренировочной выборке. На самом деле, можно было бы выкинуть ещё пару признаков, которые несут минимальное, а возможно и никакого, значения. Протестируем и сравним другие классификаторы. 

In [35]:
signs = train_X_del_params[1] # удаляем найденные признаки из выборки
train_Y = np.array(train[signs[-1]]>20) 
train_X = np.array(train[signs[:-1]])

In [36]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
# from sklearn.svm import SVC
# from sklearn.neural_network import MLPClassifier
# from sklearn.gaussian_process import GaussianProcessClassifier

classifier = [DecisionTreeClassifier, GaussianNB, KNeighborsClassifier, RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, QuadraticDiscriminantAnalysis]
classifier_name = ['DecisionTree', 'GaussianNB', 'KNeighbors', 'RandomForest', 'GradientBoosting', 'AdaBoosting', 'QuadraticDiscriminantAnalysis']
answer = pd.DataFrame(columns = ['classifier', 'test.mean()', 'train.mean()'])

for i in range(len(classifier)):
    scorer = np.array(validation(classifier[i], train_X, train_Y)) 
    answer = answer.append([{'classifier': classifier_name[i], 'test.mean()': scorer[:,0].mean(), 'train.mean()': scorer[:,1].mean()}])
    
answer.sort_values(by='test.mean()', ascending=False)    


,classifier,test.mean(),train.mean()
0,GradientBoosting,0.917497,0.935033
0,AdaBoosting,0.911620,0.927676
0,RandomForest,0.842015,0.999666
0,QuadraticDiscriminantAnalysis,0.812729,0.828034
0,GaussianNB,0.745299,0.769609
0,KNeighbors,0.700100,0.976934
0,DecisionTree,0.672665,0.999989


Сразу видно какие модели переобучаются (у них train.mean() почти 1, а вот test.mean() на кросс-валидации плохой), а какие работают исправно. При детальном рассмотрении понятно что многие модели впринцыпе не применимы к такой задаче, но приведены они для наглядности.

In [37]:
scorer = np.array(validation(AdaBoostClassifier, train_X, train_Y))
print([scorer[:,0].mean(), scorer[:,0].std(),scorer[:,1].mean(), scorer[:,1].std()])
scorer

[0.91161950010393689, 0.0050365924957506605, 0.92767596163386956, 0.0019531606381920029]


array([[ 0.91982911,  0.9246087 ],
       [ 0.90517591,  0.93074619],
       [ 0.91431999,  0.92748482],
       [ 0.90995686,  0.92807466],
       [ 0.90881562,  0.92746544]])

У AdaBoostClassifier меньше разброс, что впринцыпе может дать более устойчивые результаты. Но всё таки результат на порядок ниже.

Теперь попробуем вручную удалить признак user.description_is_website_enabled и user.geo_enabled и заново запустить алгоритм облегчения модели.

In [38]:
signs

array(['text_length', 'text_account_count', 'text_hashtag_count',
       'text_is_website_enabled', 'user.description_length',
       'user.description_hashtag_count',
       'user.description_is_website_enabled', 'in_reply_to_user_id',
       'user.location', 'user.lang', 'user.statuses_count',
       'user.followers_count', 'user.favourites_count', 'user.geo_enabled',
       'user.listed_count', 'retweet_count'], dtype=object)

In [39]:
train_X = np.delete(train_X, [5, 6], 1)
signs = np.delete(signs, [5, 6], 0)

train_X_del_params = del_params(train_X, train_Y, signs, GradientBoostingClassifier)
train_X_del_params[0]

without user.lang
without user.geo_enabled


,id,signs,test.mean(),test.std(),train.mean()
0,100.0,full,0.917545,0.006951,0.934802
0,4.0,without user.description_length,0.916863,0.006490,0.934243
0,0.0,without text_length,0.916584,0.006586,0.933661
0,2.0,without text_hashtag_count,0.916539,0.006357,0.934650
0,9.0,without user.favourites_count,0.916118,0.006183,0.933042
0,3.0,without text_is_website_enabled,0.916052,0.006879,0.934473
0,1.0,without text_account_count,0.915832,0.007043,0.933822
0,10.0,without user.listed_count,0.915769,0.007333,0.933765
0,7.0,without user.statuses_count,0.912138,0.007714,0.925890
0,6.0,without user.location,0.911620,0.006773,0.930318


# Итог

Всё равно осталось много малозначемых параметров, которые рандомно могут повернуть классификатор. Тут как повезёт, они могут подтянуть финальный результат, а могут и опустить. С уверенностю можно сказать, что на то будет набрано 20 ретвитов или нет, сильно влияет количество подписчиков пользователя и статус этого сообщения (т.е. это оригинальный твит или чей-то ретвит)

# Финальное обучение, предсказание и запись результата.

Для начала приведём тестовые данные приведём к такому же виду, что тренировочные.

In [48]:
prediction = pd.DataFrame(test['id'], index=range(len(test)), columns=['id']) # сразу выдерним id

params = text_params(test['text'])
test.insert(2, 'text_length', params[0])
test.insert(3, 'text_account_count', params[1])
test.insert(4, 'text_hashtag_count', params[2])
test.insert(5, 'text_is_website_enabled', params[3])

params = text_params(test['user.description'])
test.insert(6, 'user.description_length', params[0])
test.insert(7, 'user.description_account_count', params[1])
test.insert(8, 'user.description_hashtag_count', params[2])
test.insert(9, 'user.description_is_website_enabled', params[3])

test_X = np.array(test[signs[:-1]])

In [49]:
signs = train_X_del_params[1] # удаляем повторно найденные признаки из выборки
train_Y = np.array(train[signs[-1]]>20) 
train_X = np.array(train[signs[:-1]])

classifier = GradientBoostingClassifier() 
classifier.fit(train_X, train_Y) # обучение на всех данных тренировочной выборки

answer = classifier.predict_proba(test_X)

In [50]:
prediction.insert(1, 'probability', answer[:,1])
prediction[:5]

,id,probability
0,629692042952765440,0.004468
1,629692042717855745,0.003796
2,629692039974813696,0.007093
3,629692038242566145,0.660719
4,629692036879413248,0.005566


In [51]:
prediction.to_csv('prediction_Sivolobtsev.csv', index=False)